In [1]:
import cifar10
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import csv

In [2]:
cifar10.data_path = "data/CIFAR-10/"

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [6]:
x_train = images_train.reshape(images_train.shape[0],-1)
x_test = images_test.reshape(images_test.shape[0], -1)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)



In [9]:
x_test_scaled = sc.transform(x_test)



In [10]:
y_train = cls_train
y_test = cls_test

In [7]:
'''nsamples, nx, ny,nz = images_train.shape
train_data = images_train.reshape((nsamples,nx*ny*nz))
n_test_samples,n_test_x,n_test_y,n_test_z = images_test.shape
test_data = images_test.reshape((n_test_samples,n_test_x*n_test_y*n_test_z))

In [8]:
'''print(train_data.shape)
test_data.shape

(50000, 3072)


(10000, 3072)

In [11]:
pca = PCA()
pca.fit(x_train_scaled)



PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [12]:
pca.components_.shape

(3072, 3072)

In [14]:
k = 0
total = sum(pca.explained_variance_)
current_sum = 0
while current_sum/total < 0.95:
    current_sum += pca.explained_variance_[k]
    k += 1
k

221

In [15]:
pca = PCA(n_components = k, whiten = True)
transformed_data = pca.fit_transform(x_train_scaled)
transformed_data.shape

(50000, 221)

In [16]:
transformed_test_data = pca.transform(x_test_scaled)
transformed_test_data.shape

(10000, 221)

# Logistic

In [22]:
clf = LogisticRegression()
clf.fit(transformed_data,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
y_pred_logistic = clf.predict(transformed_test_data)
log_score = accuracy_score(y_test,y_pred_logistic)
log_score

0.4066

# random

In [24]:
clf1 = RandomForestClassifier()
clf1.fit(transformed_data,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
y_pred_random = clf1.predict(transformed_test_data)
random_score = accuracy_score(y_test,y_pred_random)
random_score

0.2731

# SVM

In [18]:
clf2 = SVC(C=100,gamma=0.005)
clf2.fit(transformed_data,y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
y_pred_svm = clf2.predict(transformed_test_data)
svc_score = accuracy_score(y_test, y_pred_svm)
svc_score

0.5689

In [26]:

final_result = []
for i in y_pred_svm:
    final_result.append(class_names[i])

In [27]:
final_result[:5]

['cat', 'ship', 'ship', 'airplane', 'cat']

In [28]:
np.savetxt("cifar10_predictions_svm.csv", final_result,delimiter = ',',fmt = '%s')